# 주유소 데이터 수집 및 분석

#### 목표 : 서울시 구별로 상표, 주유소 명, 유종별 평균 가격 (보통휘발유, 경유) 부가정보별 데이터 분석하기

In [243]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains  #동작을 하나로 묶기
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException, TimeoutException


from tqdm.notebook import tqdm
from glob import glob #파일을 확인할 수 있는!

import time
import os

In [244]:
options = webdriver.ChromeOptions()

prefs = {'download.default_directory' : '../../data/17.gas_station_price',
         'download.prompt_for_download' : False}

options.add_experimental_option('prefs' , prefs)

url = "https://www.opinet.co.kr/searRgSelect.do"
driver = webdriver.Chrome(service=Service("/home/min/dev_ws/EDA/driver/chromedriver"), #드라이버 경로는 절대경로로
                          options=options)

driver.get(url)

지역 시/도 리스트화

In [245]:
sido = driver.find_element(By.ID,"SIDO_NM0")
sido_list = sido.find_elements(By.TAG_NAME,"option")

sido_name_list= []
for idx, sido_name in enumerate(sido_list):
    print(str(idx) + ". " + sido_name.get_attribute("value"))
    sido_name_list.append(sido_name.get_attribute("value"))


0. 
1. 서울특별시
2. 부산광역시
3. 대구광역시
4. 인천광역시
5. 광주광역시
6. 대전광역시
7. 울산광역시
8. 세종특별자치시
9. 경기도
10. 강원특별자치도
11. 충청북도
12. 충청남도
13. 전북특별자치도
14. 전라남도
15. 경상북도
16. 경상남도
17. 제주특별자치도


In [246]:
sido_name_list = sido_name_list[1:]

sido_name_list

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원특별자치도',
 '충청북도',
 '충청남도',
 '전북특별자치도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [247]:
len(sido_name_list)

17

시/도 dropbox -> 서울시로 지역 지정

In [248]:
print(sido_name_list[0])
sido.send_keys(sido_name_list[0])

서울특별시


시/군/구 리스트화

In [249]:
gu = driver.find_element(By.ID,"SIGUNGU_NM0")
gu_list = gu.find_elements(By.TAG_NAME,"option")

gu_name_list= []
for idx, gu_name in enumerate(gu_list):
    print(str(idx) + ". " + gu_name.get_attribute("value"))
    gu_name_list.append(gu_name.get_attribute("value"))

0. 
1. 강남구
2. 강동구
3. 강북구
4. 강서구
5. 관악구
6. 광진구
7. 구로구
8. 금천구
9. 노원구
10. 도봉구
11. 동대문구
12. 동작구
13. 마포구
14. 서대문구
15. 서초구
16. 성동구
17. 성북구
18. 송파구
19. 양천구
20. 영등포구
21. 용산구
22. 은평구
23. 종로구
24. 중구
25. 중랑구


In [250]:
gu_name_list = gu_name_list[1:]

In [251]:
len(gu_name_list)

25

### 주유소 정보 가져오기
#### 1. 연료종류별 주유소 정보

- 연료 탭 이름 리스트화

In [252]:
gu.send_keys(gu_name_list[1])

In [253]:

fuel = driver.find_element(By.ID,"templ_list0") #연료정보 보관 ID
# fuel = driver.find_elements(By.CLASS_NAME,"tab_type2 mgb_7") class로는 왜 안찾아질까?

tab_list = fuel.find_elements(By.TAG_NAME,"li") #연료 종류 tab 클릭
tab_list[1].click() #보통휘발유 

In [254]:
fuel_name_list=[]
for a in tab_list:
    tag_name=a.find_element(By.TAG_NAME,"a").get_attribute("title")
    fuel_name_list.append(tag_name)

fuel_name_list

['고급휘발유', '보통휘발유', '경유', '실내등유']

1. 보통 휘발유 주유소 팝업 주소 리스트화

In [257]:
gasoline = fuel.find_elements(By.ID,"os_price1")
hi_gasoline = fuel.find_elements(By.ID,"os_price2")
disel = fuel.find_elements(By.ID,"os_price3")
lpg = fuel.find_elements(By.ID,"os_price4")

In [258]:
# javascript_links = gasoline.find_elements(By.XPATH, '//a[contains(@href, "javascript:fn_osPop")]') 
# 오류 이유 : find elements는 웹드라이버 요소에 적용가능하다. list에는 적용 불가, 따라서 for문으로 반복해야한다.

gsl_link_list=[]
for href in gasoline:
    try:
        pop_ups = href.find_elements(By.XPATH, './/a[contains(@href, "javascript:fn_osPop")]')
        # 이때!! find_elements!!!!! find_element 하면 하나만 찾아짐
        gsl_link_list.extend(pop_ups)
        #append 가 아니라 extend로 넣어야 됨 append는 찾은거를 하나의 요소로 때려넣음 extendsms 하나하나씩 넣어줌
    except Exception as e:
        # 보통 휘발유에 주유소가 없을경우
        print("NO GAS STATION:", str(e))


링크 개수 확인(강동구 보통 휘발유 주유소 총 14개)

In [259]:
gsl_link_list
len(gsl_link_list)


14

링크가 뜨는지 확인

In [260]:
gsl_link_list[2].click()

2. 지도에서 정보 가져오기  

- 기본 정보 : 주유소 이름, 전화번호, 주소, 상표  
보통 휘발유로 먼저 확인

In [261]:
info = driver.find_element(By.ID,"os_dtail_info")

st_name = info.find_element(By.ID,"os_nm").text
gsl_st = info.find_element(By.CLASS_NAME,"gis_detail_info_bcon").find_elements(By.TAG_NAME,"label")

gsl_st_info = [label.text for label in gsl_st]
gsl_st_info.insert(0, st_name)

gsl_st_info

['', '02-442-5145', '서울 강동구 동남로 811 (명일동)', 'SK에너지']

- 유가 정보 : 유종, 가격

In [262]:
gsl_cost = info.find_element(By.ID,"b027Div")
gsl_cost_info = [gsl_cost.find_element(By.CLASS_NAME,"nobd_l").text, gsl_cost.find_element(By.ID,"b027_p").text]

gsl_cost_info

['보통휘발유', '1,665']

- 부가 정보 : 세차장, 충전소, 경정비, 편의점, 24시영업

##### 부가정보 여부가 담겨있는 곳 : image file 이름  
유무 확인 방법 : image 파일 이름 뒤 확인  
-> '숫자'  무 -> 'off'  
------
    -> 하기 주유소는 세차장만 서비스 제공

In [263]:
car_wash = info.find_element(By.ID,"cwsh_yn").get_attribute('src')
charging = info.find_element(By.ID,"lpg_yn").get_attribute('src')
maint = info.find_element(By.ID,"maint_yn").get_attribute('src')
cvs = info.find_element(By.ID,"cvs_yn").get_attribute('src')
open_24 = info.find_element(By.ID,"sel24_yn").get_attribute('src')

#세차장만 있음

car_wash,charging,maint,cvs,open_24

('https://www.opinet.co.kr/images/user/gis/oil_station_service1_01.gif',
 'https://www.opinet.co.kr/images/user/gis/oil_station_service1_02_01_off.gif',
 'https://www.opinet.co.kr/images/user/gis/oil_station_service1_03_off.gif',
 'https://www.opinet.co.kr/images/user/gis/oil_station_service1_04_off.gif',
 'https://www.opinet.co.kr/images/user/gis/oil_station_service1_05_off.gif')

In [264]:
serv_car = car_wash.split('.')[-2][-3:]
serv_chr = charging.split('.')[-2][-3:]
serv_maint = maint.split('.')[-2][-3:]
serv_cvs = cvs.split('.')[-2][-3:]
serv_open_24 = open_24.split('.')[-2][-3:]

serve = [serv_car, serv_chr, serv_maint, serv_cvs, serv_open_24]

for idx in range(len(serve)):
    if serve[idx] == 'off':
        serve[idx] = "X"
    else : serve[idx] = "O"

serve



['O', 'X', 'X', 'X', 'X']

tab 순서랑 id 순서랑 맞춰준다


In [265]:
temp = fuel_name_list[0]
fuel_name_list[0] = fuel_name_list[1]
fuel_name_list[1] = temp

fuel_name_list

['보통휘발유', '고급휘발유', '경유', '실내등유']

# 합칠꺼다!!

In [131]:
total_info_list = []

for gu_click in gu_name_list: #구 선택
    gu = driver.find_element(By.ID, "SIGUNGU_NM0")  # Replace with the appropriate locator
    gu.send_keys(gu_click)

    
    for title in fuel_name_list:#휘발유 종류 선택
        tab= driver.find_element(By.XPATH, f'.//a[contains(@title, "{title}")]') 
        tab.click()
        time.sleep(0.1)

        # total_info = [gu_click,title]
        # total_info_list.append(total_info)

        

total_info_list

[['강남구', '보통휘발유'],
 ['강남구', '고급휘발유'],
 ['강남구', '경유'],
 ['강남구', '실내등유'],
 ['강동구', '보통휘발유'],
 ['강동구', '고급휘발유'],
 ['강동구', '경유'],
 ['강동구', '실내등유'],
 ['강북구', '보통휘발유'],
 ['강북구', '고급휘발유'],
 ['강북구', '경유'],
 ['강북구', '실내등유'],
 ['강서구', '보통휘발유'],
 ['강서구', '고급휘발유'],
 ['강서구', '경유'],
 ['강서구', '실내등유'],
 ['관악구', '보통휘발유'],
 ['관악구', '고급휘발유'],
 ['관악구', '경유'],
 ['관악구', '실내등유'],
 ['광진구', '보통휘발유'],
 ['광진구', '고급휘발유'],
 ['광진구', '경유'],
 ['광진구', '실내등유'],
 ['구로구', '보통휘발유'],
 ['구로구', '고급휘발유'],
 ['구로구', '경유'],
 ['구로구', '실내등유'],
 ['금천구', '보통휘발유'],
 ['금천구', '고급휘발유'],
 ['금천구', '경유'],
 ['금천구', '실내등유'],
 ['노원구', '보통휘발유'],
 ['노원구', '고급휘발유'],
 ['노원구', '경유'],
 ['노원구', '실내등유'],
 ['도봉구', '보통휘발유'],
 ['도봉구', '고급휘발유'],
 ['도봉구', '경유'],
 ['도봉구', '실내등유'],
 ['동대문구', '보통휘발유'],
 ['동대문구', '고급휘발유'],
 ['동대문구', '경유'],
 ['동대문구', '실내등유'],
 ['동작구', '보통휘발유'],
 ['동작구', '고급휘발유'],
 ['동작구', '경유'],
 ['동작구', '실내등유'],
 ['마포구', '보통휘발유'],
 ['마포구', '고급휘발유'],
 ['마포구', '경유'],
 ['마포구', '실내등유'],
 ['서대문구', '보통휘발유'],
 ['서대문구', '고급휘발유'],
 ['서대문구', '경유'],
 ['

In [132]:
len(total_info_list)

100

한 구에 있는 고급휘발유/보통휘발유/경유/실내등유 제공 주유소 정보 가져오기

pop up 다 누를 수 있는 상태인가? ?? ?? yes!!

In [144]:


link_list=[]
for idx in range(len(fuel_name_list)):
    gas_station = driver.find_element(By.ID,f"os_price{idx+1}")
    pop_ups = gas_station.find_elements(By.XPATH, './/a[contains(@href, "javascript:fn_osPop")]')

    link_list.extend(pop_ups)
    
len(link_list),link_list


(37,
 [<selenium.webdriver.remote.webelement.WebElement (session="f17f60dc3b7e84839e1f873e22988b64", element="f.FBCF0DD899DAE131C6A47F7246959674.d.7447F13A98376D924825EA102542C3AE.e.42958")>,
  <selenium.webdriver.remote.webelement.WebElement (session="f17f60dc3b7e84839e1f873e22988b64", element="f.FBCF0DD899DAE131C6A47F7246959674.d.7447F13A98376D924825EA102542C3AE.e.42984")>,
  <selenium.webdriver.remote.webelement.WebElement (session="f17f60dc3b7e84839e1f873e22988b64", element="f.FBCF0DD899DAE131C6A47F7246959674.d.7447F13A98376D924825EA102542C3AE.e.43007")>,
  <selenium.webdriver.remote.webelement.WebElement (session="f17f60dc3b7e84839e1f873e22988b64", element="f.FBCF0DD899DAE131C6A47F7246959674.d.7447F13A98376D924825EA102542C3AE.e.43030")>,
  <selenium.webdriver.remote.webelement.WebElement (session="f17f60dc3b7e84839e1f873e22988b64", element="f.FBCF0DD899DAE131C6A47F7246959674.d.7447F13A98376D924825EA102542C3AE.e.43053")>,
  <selenium.webdriver.remote.webelement.WebElement (session=

In [198]:
# 안되는 이유 idx 가 따로 같이 커지면 안됨!! 탭 바뀔때만 바뀌어야 한다굿!!!!

# total_info_list = []

# for gu_click in gu_name_list: #구 선택
#     gu = driver.find_element(By.ID, "SIGUNGU_NM0")  # Replace with the appropriate locator
#     gu.send_keys(gu_click)

    
#     for title in fuel_name_list:#휘발유 종류 선택
#         tab= driver.find_element(By.XPATH, f'.//a[contains(@title, "{title}")]') 
#         tab.click()
        
#         pop_up_list = []
#         for idx in range(len(fuel_name_list)):
#             gas_station = driver.find_element(By.ID,f"os_price{idx+1}")
#             pop_up = gas_station.find_elements(By.XPATH, './/a[contains(@href, "javascript:fn_osPop")]')
            
#             pop_up_list.extend(pop_up)
            
#         for pop_click in pop_up_list:
#             gu = driver.find_element(By.ID, "SIGUNGU_NM0")  # Replace with the appropriate locator
#             pop_click.click()



In [172]:
len(pop_up_list)

98

In [197]:
# 안돌아가는 이유 pop up list 초기화 안해서


# # total_info_list = []
# # title=[]
# # pop_up_list=[]

# # for gu_click in gu_name_list: #구 선택
# #     gu = driver.find_element(By.ID, "SIGUNGU_NM0")  # Replace with the appropriate locator
# #     gu.send_keys(gu_click)

# #     idx = 1
# #     for  title in fuel_name_list:#휘발유 종류 선택
    
# #         tab= driver.find_element(By.XPATH, f'.//a[contains(@title, "{title}")]') 
# #         tab.click()
        
# #         gas_station = driver.find_element(By.ID,f"os_price{idx}")
# #         idx += 1
# #         pop_up = gas_station.find_elements(By.XPATH, './/a[contains(@href, "javascript:fn_osPop")]')
            
# #         pop_up_list.extend(pop_up)
        
    
# #         for pop_click in pop_up_list:
# #             gu = driver.find_element(By.ID, "SIGUNGU_NM0")  # Replace with the appropriate locator
# #             pop_click.click()
# #             time.sleep(0.2)

In [186]:
len(pop_up_list)

60

In [194]:
idx

2

In [196]:
total_info_list = []

for gu_click in gu_name_list: # 구 선택
    gu = driver.find_element(By.ID, "SIGUNGU_NM0")  # Replace with the appropriate locator
    gu.send_keys(gu_click)

    for idx, title in enumerate(fuel_name_list, start=1):  # 휘발유 종류 선택
        tab = driver.find_element(By.XPATH, f'.//a[contains(@title, "{title}")]') 
        tab.click()

        gas_station = driver.find_element(By.ID, f"os_price{idx}")
        pop_up_list = gas_station.find_elements(By.XPATH, './/a[contains(@href, "javascript:fn_osPop")]')

        for pop_click in pop_up_list:
            try:
                pop_click.click()
                time.sleep(0.2)
            except ElementNotInteractableException:
                print("Element not interactable. Skipping click.")

            # Add your logic to handle the pop-up here

            # After handling the pop-up, you may need to switch back to the main window
            driver.switch_to.window(driver.window_handles[0])

            # Optionally, you can append relevant information to total_info_list


In [203]:
# total_info_list = []
# for idx, fuel_kind in enumerate(fuel_name_list, start=1):  # 휘발유 종류 선택
#         tab = driver.find_element(By.XPATH, f'.//a[contains(@title, "{fuel_kind}")]') 
#         tab.click()

#         gas_station = driver.find_element(By.ID, f"os_price{idx}")
#         pop_up_list = gas_station.find_elements(By.XPATH, './/a[contains(@href, "javascript:fn_osPop")]')

#         for pop_click in pop_up_list:
#             try:
#                 pop_click.click()
#                 info = driver.find_element(By.ID,"os_dtail_info")

#                 st_name = info.find_element(By.ID,"os_nm").text
#                 st = info.find_element(By.CLASS_NAME,"gis_detail_info_bcon").find_elements(By.TAG_NAME,"label")

#                 st_info = [label.text for label in st]


#                 total_info = [gu_click,title,st_name,st_info]
#                 total_info_list.append(total_info)

#             except ElementNotInteractableException:
#                 print("Element not interactable. Skipping click.")

#             # Add your logic to handle the pop-up here

#             # After handling the pop-up, you may need to switch back to the main window
#             driver.switch_to.window(driver.window_handles[0])

#             # Optionally, you can append relevant information to total_info_list


이 코드는 정보를 검색하기 전에 팝업 콘텐츠의 존재를 기다리는 것을 포함하며 팝업 상호 작용 중에 예외를 처리합니다. 특정 시나리오에 맞게 필요에 따라 조정합니다. 그래도 안나온다고!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "os_dtail_info")))
            except TimeoutException:
                print("Timed out waiting for pop-up details.")

In [267]:
total_info_list = []

for gu_click in gu_name_list: # 구 선택
    gu = driver.find_element(By.ID, "SIGUNGU_NM0")  # Replace with the appropriate locator
    gu.send_keys(gu_click)

    for idx, fuel_kind in enumerate(fuel_name_list, start=1):  # 휘발유 종류 선택
            tab = driver.find_element(By.XPATH, f'.//a[contains(@title, "{fuel_kind}")]') 
            tab.click()

            gas_station = driver.find_element(By.ID, f"os_price{idx}")
            pop_up_list = gas_station.find_elements(By.XPATH, './/a[contains(@href, "javascript:fn_osPop")]')

            for pop_click in pop_up_list:
                try:
                    pop_click.click()
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "os_dtail_info")))

                    info = driver.find_element(By.ID,"os_dtail_info")

                    st_name = info.find_element(By.ID,"os_nm").text
                    st = info.find_element(By.CLASS_NAME,"gis_detail_info_bcon").find_elements(By.TAG_NAME,"label")

                    st_info = [label.text for label in st]

                    total_info = [gu_click,fuel_kind,st_name,st_info]
                    total_info_list.append(total_info)

                except ElementNotInteractableException:
                    print("Element not interactable. Skipping click.")
                    
                except TimeoutException:
                    print("Timed out waiting for pop-up details.")

                # Add your logic to handle the pop-up here

                # After handling the pop-up, you may need to switch back to the main window
                driver.switch_to.window(driver.window_handles[0])

                # Optionally, you can append relevant information to total_info_list


In [268]:
st_name

''

In [269]:
total_info_list

[['강남구', '보통휘발유', '', ['', '', 'HD현대오일뱅크']],
 ['강남구', '보통휘발유', '', ['', '', '']],
 ['강남구', '보통휘발유', '', ['', '', '']],
 ['강남구', '보통휘발유', '', ['', '', '']],
 ['강남구', '보통휘발유', '', ['', '', '']],
 ['강남구', '보통휘발유', '', ['02-445-5841', '서울 강남구 밤고개로 120 (자곡동)', 'SK에너지']],
 ['강남구', '보통휘발유', '', ['', '', '']],
 ['강남구', '보통휘발유', '', ['', '서울 강남구 밤고개로 215 (율현동)', 'GS칼텍스']],
 ['강남구', '보통휘발유', '', ['', '', '']],
 ['강남구', '보통휘발유', '', ['02-2226-5188', '서울 강남구 양재대로55길 3 (일원동)', 'S-OIL']],
 ['강남구', '보통휘발유', '', ['02-562-4855', '서울 강남구 역삼로 142', 'SK에너지']],
 ['강남구', '보통휘발유', '', ['02-2058-3024', '서울 강남구 도곡로 162 (도곡동)', 'S-OIL']],
 ['강남구', '보통휘발유', '', ['', '', '']],
 ['강남구', '보통휘발유', '', ['', '', '']],
 ['강남구', '보통휘발유', '', ['', '', '']],
 ['강남구', '보통휘발유', '', ['02-539-5145', '서울 강남구 논현로 516 (역삼동)', 'GS칼텍스']],
 ['강남구', '보통휘발유', '', ['02-545-6161', '서울 강남구 언주로 842 (신사동)', 'SK에너지']],
 ['강남구', '보통휘발유', '', ['', '', '']],
 ['강남구', '보통휘발유', '', ['02-573-9031', '서울 강남구 양재대로 339', 'GS칼텍스']],
 ['강남구', '보통휘발유',

In [271]:
total_info_list = []

for gu_click in gu_name_list: # 구 선택
    gu = driver.find_element(By.ID, "SIGUNGU_NM0")  # Replace with the appropriate locator
    gu.send_keys(gu_click)

    for idx, fuel_kind in enumerate(fuel_name_list, start=1):  # 휘발유 종류 선택
        tab = driver.find_element(By.XPATH, f'.//a[contains(@title, "{fuel_kind}")]') 
        tab.click()

        gas_station = driver.find_element(By.ID, f"os_price{idx}")
        pop_up_list = gas_station.find_elements(By.XPATH, './/a[contains(@href, "javascript:fn_osPop")]')

        for pop_click in pop_up_list:
            try:
                pop_click.click()
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "os_dtail_info")))

                info = driver.find_element(By.ID,"os_dtail_info")

                st_name = info.find_element(By.ID,"os_nm").text
                st = info.find_element(By.CLASS_NAME,"gis_detail_info_bcon").find_elements(By.TAG_NAME,"label")

                st_info = [label.text for label in st]

                total_info = [gu_click,fuel_kind,st_name,st_info]
                total_info_list.append(total_info)

            except ElementNotInteractableException:
                print("Element not interactable. Skipping click.")
                
            except TimeoutException:
                print("Timed out waiting for pop-up details.")

            finally:
                # Close the pop-up using WebDriverWait
                try:
                    close_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "pop_map_close")))
                    close_button.click()
                except TimeoutException:
                    print("Timed out waiting for the close button.")

Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.
Timed out waiting for the close button.


KeyboardInterrupt: 

이 수정은 팝업 요소의 가시성을 명시적으로 기다리는 기능을 제공합니다. 더 많은 안정성을 제공해야 합니다. 이것을 시도해 보시고 문제가 해결되면 알려주세요.  This modification introduces an explicit wait for the visibility of the pop-up elements. It should provide more stability. Please try this and let me know if it resolves the issue. 쥰나 오래걸림


If the delay caused by waiting for the close button is too long, we can try a different approach. Instead of explicitly closing the pop-up, we can attempt to switch back to the main window after extracting the information we need from the pop-up.

Here is the modified code: